In [1]:
!pwd

/Users/ariellubonja/prog/Leiden-GEE-work/leiden-runtime-experiments


In [2]:
%cd ..

/Users/ariellubonja/prog/Leiden-GEE-work


# Using Real Graphs -  loading is pretty slow

In [ ]:
import networkx as nx
import igraph as ig
import leidenalg

# G = np.loadtxt("carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", delimiter=',')
graph = nx.read_weighted_edgelist("carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", delimiter=",")

G = ig.Graph.from_networkx(graph)

In [ ]:
%%timeit # Time how long it takes to load graph

nx.read_weighted_edgelist("carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", delimiter=",")
ig.Graph.from_networkx(graph)

### Largest

In [5]:
%%timeit

partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition)

22.4 s ± 3.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Enron

In [ ]:
%%timeit # Time how long it takes to load graph

nx.read_weighted_edgelist("carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", delimiter=",")
ig.Graph.from_networkx(graph)

In [8]:
enron = ig.Graph.from_networkx(nx.read_weighted_edgelist("carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", delimiter=","))

In [9]:
%%timeit

partition = leidenalg.find_partition(enron, leidenalg.ModularityVertexPartition)

6.76 µs ± 54.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### n500

In [ ]:
%%timeit # Time how long it takes to load graph

nx.read_weighted_edgelist("carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", delimiter=",")
ig.Graph.from_networkx(graph)

In [11]:
n500 = ig.Graph.from_networkx(nx.read_weighted_edgelist("carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", delimiter=","))

In [12]:
%%timeit

partition = leidenalg.find_partition(n500, leidenalg.ModularityVertexPartition)

27.8 ms ± 544 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Networkit:ParallelLeiden

### MSR graph - 1.2GB

In [ ]:
!pip3 install networkit

In [ ]:
import networkit as nk

In [ ]:
# Not sure why this library wants to know the ID of the first node. It's required arg
G = nk.graphio.readGraph("carey-tsg-graphs/edgelist-msr-n32277-T24-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=1)

print(G.numberOfNodes(), "nodes")
print(G.numberOfEdges(), "edges")

In [ ]:
%%timeit
pl = nk.community.ParallelLeiden(G)

pl.run()

In [ ]:
help(nk.community.ParallelLeiden)

In [ ]:
partition = pl.getPartition()

In [ ]:
partition.subsetSizes()

### Enron graph

In [ ]:
# import numpy as np

# G = np.loadtxt("carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", delimiter=',', dtype=int)

# Find firstNode ID - needed for Networkit
# print(np.min(G[:,0]))
# print(np.max(G[:,1]))

In [ ]:
# Firstnode ID has to be correct! 0 for enron
G = nk.graphio.readGraph("carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=0)

print(G.numberOfNodes(), "nodes")
print(G.numberOfEdges(), "edges")

In [ ]:
%%timeit
pl = nk.community.ParallelLeiden(G)

pl.run()

### n500

In [ ]:
# import numpy as np

# G = np.loadtxt("carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", delimiter=',')#, dtype=int)

# print(np.min(G[:,0])) # Find First Node ID
# print(np.max(G[:,1]))

In [ ]:
G = nk.graphio.readGraph("carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", nk.Format.EdgeList, separator=',', firstNode=1)

print(G.numberOfNodes(), "nodes")
print(G.numberOfEdges(), "edges")

In [ ]:
%%timeit
pl = nk.community.ParallelLeiden(G)

pl.run()

## Graph Encoder Embedding

#### The below sometimes works and sometimes ModuleNotFound - why?

In [ ]:
# This is necessary for GraphEmbedding to work!
%cd ~/prog/Leiden-GEE-work/GraphEmd/

In [ ]:
!ls Main/Python

In [ ]:
# absolute_path = '/users/ariel/prog/'
absolute_path = '/Users/ariellubonja/prog/Leiden-GEE-work/'
# graph_path = '/static/ariel'
graph_path = absolute_path + '/carey-tsg-graphs'

# Change this depending on your folder structure

In [ ]:
%cd ~/prog

In [ ]:
import sys
sys.path.append(absolute_path + 'GraphEmd/Main/Python')
sys.path.append(absolute_path + 'GraphEmd')
import Evaluation # GEE's Evaluation.py

### MSR 1.2GB

In [ ]:
%%timeit

Evaluation.loadGraph(graph_path + "/edgelist-msr-n32277-T24-forRB2.csv", weighted=True, randomY=True)

In [ ]:
G_edgelist, Y, n = Evaluation.loadGraph(graph_path + "/edgelist-msr-n32277-T24-forRB2.csv", weighted=True, randomY=True)

In [ ]:
%%timeit
Z, W = Evaluation.graph_encoder_embed(G_edgelist, Y, n, Correlation = False, Laplacian = False)

### Enron

In [ ]:
G_edgelist, Y, n = Evaluation.loadGraph(graph_path + "/carey-tsg-graphs/edgelist-enron-n184-forRB2.csv", weighted=True, randomY=True)

In [ ]:
%%timeit
Z, W = Evaluation.graph_encoder_embed(G_edgelist, Y, n, Correlation = False, Laplacian = False)

In [ ]:
!ls /static/ariel/carey-tsg-graphs

### n500

In [ ]:
G_edgelist, Y, n = Evaluation.loadGraph(graph_path + "/carey-tsg-graphs/edgelist-n500-T40-forRB2.csv", weighted=True, randomY=True)

In [ ]:
%%timeit
Z, W = Evaluation.graph_encoder_embed(G_edgelist, Y, n, Correlation = False, Laplacian = False)